<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install deep-daze stomp.py --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)


In [28]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
 
flatten = itertools.chain.from_iterable

CMD = 'imagine --lower_bound_cutout 0.7 --deeper --image-width=384 --iterations=500 --epochs=5 --save-every=15 --save-progress=True --open_folder=False --overwrite=True "An intricate drawing of an individual alien Neuron"'
WATCH_MEDIA_FILES = ["./*.jpg","./*.mp4"]

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)


!rm *.jpg

p = Popen(CMD, stdout = PIPE, 
        stderr = STDOUT, shell = True)

FOUND_MEDIA_FILES=[]
while True:
  line = p.stdout.readline()#
  if not line: break

  stdout.write(line)
  conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
  NEW_FOUND_MEDIA_FILES=list(flatten([glob(search_path) for search_path in WATCH_MEDIA_FILES]))
  NEW_FOUND_MEDIA_FILES.sort(key=os.path.getmtime)
  ADDED_MEDIA_FILES = list(set(NEW_FOUND_MEDIA_FILES)-set(FOUND_MEDIA_FILES))
  FOUND_MEDIA_FILES = NEW_FOUND_MEDIA_FILES

  if len(ADDED_MEDIA_FILES) > 0:
    for added in ADDED_MEDIA_FILES:
      sleep(0.1)
      if len(added) > 0:
        print("reading", added)
        with open(added,"rb") as added_media:
          base64_encoded_added = f"data:image/jpg;base64,{b64encode(added_media).decode('ascii')}"
        
        conn.send(body=base64_encoded_added, destination='/topic/colabMediaOut')
conn.disconnect()

rm: cannot remove '*.jpg': No such file or directory
epochs:   0%|          | 0/5 [00:00<?, ?it/s]
                                             
reading ./An_intricate_drawing_of_an_individual_alien_Neuron.jpg
loss: -37.45:   3%|▎         | 14/500 [00:26<14:09,  1.75s/it]
                                             
reading ./An_intricate_drawing_of_an_individual_alien_Neuron.000001.jpg
loss: -44.29:   6%|▌         | 29/500 [00:52<13:43,  1.75s/it]
                                             
reading ./An_intricate_drawing_of_an_individual_alien_Neuron.000002.jpg
loss: -47.23:   9%|▉         | 44/500 [01:18<13:17,  1.75s/it]
                                             
reading ./An_intricate_drawing_of_an_individual_alien_Neuron.000003.jpg
loss: -49.99:  12%|█▏        | 59/500 [01:44<12:50,  1.75s/it]
                                             
reading ./An_intricate_drawing_of_an_individual_alien_Neuron.000004.jpg
loss: -51.94:  15%|█▍        | 74/500 [02:11<12:25,  1.75s/it]
   

KeyboardInterrupt: ignored